In [4]:
import requests
import re
import time
import csv
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
from py2neo import Node, Relationship, Graph


exist_url = []
g_writecount = 0

#================实体们的表====================
properties_from = ['id','name','LABEL']
properties_to=['id','name','LABEL']
properties_relation=['START_ID','END_ID','TYPE']
f = open('properties_from.csv', 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(properties_from)
f = open('properties_to.csv', 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(properties_to)
f = open('properties_relation.csv', 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(properties_relation)

    
    
#================类别们的表====================
concept_from = ['id','name','LABEL']
concept_to=['id','name','LABEL']
concept_relation=['START_ID','END_ID','TYPE']
f = open('concept_from.csv', 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(properties_from)
f = open('concept_to.csv', 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(properties_to)
f = open('concept_relation.csv', 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(properties_relation)
    
    
    
#====================伪装成浏览器==============    
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) \
                   Chrome/75.0.3770.142 Safari/537.36"
                   }


#================用来在实体页面抽取infobox的函数============== 
def scrappy_tar(url,father):
        tmp=url     
        global headers
        url = 'https://www.wanweibaike.com/wiki-' + url    
        properties_from=[tmp,tmp,father]
        f = open('properties_from.csv', 'a')
        with f:
                writer = csv.writer(f)
                writer.writerow(properties_from)
        html = requests.get(url, headers=headers,timeout=1).text
        soup = BeautifulSoup(html, 'lxml',multi_valued_attributes=None)
        try:
            table = soup.find('table', class_=re.compile('infobox'))#找到infobox，只要类名里面有infobox的皆要
            trs = table.find_all('tr')#infobox中的每一行是tr
            res_list=[]
            concept=[]
            entity=[]
            for i in trs:
                    i=str(i).replace('< a','<a').replace('/ a','/a')
                    soup = BeautifulSoup(i.strip())
                    a=['None']
                    i = soup.find('th')#关系名
                    #i=datawashing(str(i))#数据清洗
                    if i:
                        a=[i.get_text()]
                    concept.append(a)#添加关系名
                    a=['None']
                    i = soup.find('td')#关系指向的实体
                    i=str(i)
                    #i=datawashing(str(i))#数据清洗
                    #=========把指向的所有实体存成一个列表============
                    if '<a'not in i.strip():#没有链接的文本
                            soup = BeautifulSoup(i.strip())
                            a=[soup.get_text()]
                    else:#有链接的文本
                            soup = BeautifulSoup(i.strip())
                            a=[i2.get_text() for i2 in soup.find_all('a')]
                    if a==['']:
                        a=["None"]
                    entity.append(a)#添加关系指向的实体
            newc=[]#关系名
            newe=[]#关系指向的实体
            #========大清洗==========
            for i in range(len(concept)):
                if concept[i]!=['None'] and entity[i]!=['None']:
                    newc.append(concept[i])
                    newe.append(entity[i])
            #========把得到的关系名和关系指向的实体存进csv===============
            for i in range(len(newe)):
                for j in range(len(newe[i])):
                    if newe[i][j]!='' and newc[i][0]!='':
                        properties=[tmp,newe[i][j],newc[i][0]]
                        f = open('properties_relation.csv', 'a')
                        with f:
                            writer = csv.writer(f)
                            writer.writerow(properties)
            for i in range(len(newe)):
                for j in range(len(newe[i])):
                    if newe[i][j]!='':
                        properties_to=[newe[i][j],newe[i][j],newe[i][j]]
                        f = open('properties_to.csv', 'a')
                        with f:
                            writer = csv.writer(f)
                            writer.writerow(properties_to)
        except:
            print("no info box",url)
            pass
        
#爬取类别页面
def scrappy(url, max_depth=2,depth=1,path=['Category-军事']):
    c_from=url
    global g_writecount,exist_url,headers
    exist_url.append(url)#防止成环
    try:
        concept_from=[url.replace(':','-'),url.replace(':','-'),'concept']
        url = 'https://www.wanweibaike.com/wiki/' + url
        #print(url)      
        f = open('concept_from.csv', 'a')
        with f:
            writer = csv.writer(f)
            writer.writerow(concept_from)
        r = requests.get(url, headers=headers)#用get方法获取页面
        html = r.text#获取页面内容
    except Exception as e:
        print("Failed downloading and saving ", url)
        print(e)
        exist_url.append(url)
        return None
    
    
    link_list = re.findall('<a href="/wiki/([^:#=<>]*?)".*?title="(Category:.*?)">.*</a>', html)#使用正则表达式找出页面中所有链接
    link_list = link_list[:len(link_list)-1]
    tmp=[]
    for i in link_list: 
        if '分类' not in i[1]:
            tmp.append(i[1])            
    link_list=tmp
    unique_list = list(set(link_list) - set(exist_url))#防止成环，找出没爬过的新链接
  

    #全都是实体页面   
    if unique_list==[]:
        link_list = re.findall('<a href="/wiki-(.*?)".*?title="(.*?)">.*</a>', html)#使用正则表达式找出实体页面的所有链接
        tmp=[]
        for i in link_list: 
            if '分类' not in i[1]:
                tmp.append(i[1])            
        link_list=tmp
        unique_list = list(set(link_list) - set(exist_url))#防止成环，找出没爬过的新链接
        for eachone in unique_list:
            if 1:#限制跳数
                a=path.copy()
                a.append(eachone)
                c=tree          
                for i in range(len(a)):
                    try:
                        c=c[a[i]]
                    except:
                        c[a[i]]={}
                try:
                    scrappy_tar(eachone.replace(':','-'),c_from)
                except:
                    pass
                time.sleep(1)
    
    
    
    
    
    
    
    
    
    
    #全都是分类页面
    for eachone in unique_list:
        if depth<max_depth:#限制跳数
            a=path.copy()
            a.append(eachone)
            c=tree
            
            for i in range(len(a)):
                    try:
                        c=c[a[i]]
                    except:
                        c[a[i]]={}
               
            #print(eachone[1])
            concept_to=[eachone.replace(':','-'),eachone.replace(':','-'),'concept']
            f = open('concept_to.csv', 'a')
            with f:
                writer = csv.writer(f)
                writer.writerow(concept_to)
            relation=[c_from.replace(':','-'),eachone.replace(':','-'),'include']
            f = open('concept_relation.csv', 'a')
            with f:
                writer = csv.writer(f)
                writer.writerow(relation)
            #print(eachone)
            time.sleep(0.5)
            try:
                scrappy(eachone.replace(':','-'), max_depth,depth+1,a)
            except:
                pass
start = time.time()
tree={}
scrappy('Category-第二次世界大战中的欧洲',6)
print(tree)#存树状结构
stop = time.time()
print("深度优先的递归爬虫所用时间：", stop-start)

#链接自己的neo4j图谱，记得用自己的账户和密码
graph = Graph("bolt://localhost:7687", password='114514')
#读取文件数据
dpt=pd.read_csv('concept_to.csv')
dpf=pd.read_csv('concept_from.csv')
df = pd.read_csv('concept_relation.csv')
count=0
l=df['START_ID'].unique()
exist=[]
for i in l:
    nf=dpf[dpf['name']==i]['name'].iloc[0]
    count+=1
    node_from = Node(nf,name = i,id = count,label= 'concept')
    graph.merge(node_from,'name','name')#先按a整合，再按b整合
    nt=df[df['START_ID']==i]    
    m=nt['END_ID'].unique()
    for x in m:
        if 'Category-' not in x:
            count+=1
            node_to = Node(x,name = dpt[dpt['name']==x]['name'].iloc[0],id = count,label= x)
        else:
            count+=1
            node_to = Node(x,name = x,id = count,label= 'concept')
        graph.merge(node_to,'name','name')
        ab = Relationship(node_from, df[df['END_ID']==x]['TYPE'].iloc[0], node_to)
        graph.merge(ab)


dpt=pd.read_csv('properties_to.csv')
dpf=pd.read_csv('properties_from.csv')
df = pd.read_csv('properties_relation.csv')
l=df['START_ID'].unique()
for i in l:
    nf=dpf[dpf['name']==i]['LABEL'].iloc[0]
    count+=1
    node_from = Node(nf,name = i,id = count,label= nf)
    graph.merge(node_from,'name','name')  
    nt=df[df['START_ID']==i]
    m=nt['END_ID'].unique()
    for x in m:
        count+=1
        node_to = Node('属性',name = dpt[dpt['name']==x]['name'].iloc[0],id = count,label= '属性')
        graph.merge(node_to,'name','name')
        ab = Relationship(node_from, df[df['END_ID']==x]['TYPE'].iloc[0], node_to)
        graph.merge(ab)



no info box https://www.wanweibaike.com/wiki-消灭法西斯，自由属于人民！
no info box https://www.wanweibaike.com/wiki-安东尼耶·伊沙科维奇
no info box https://www.wanweibaike.com/wiki-南斯拉夫人民解放军和游击队
no info box https://www.wanweibaike.com/wiki-8.8厘米KwK 36炮
no info box https://www.wanweibaike.com/wiki-8.8厘米KwK 43炮
no info box https://www.wanweibaike.com/wiki-5厘米KwK 39炮
no info box https://www.wanweibaike.com/wiki-7.5厘米KwK 40炮
no info box https://www.wanweibaike.com/wiki-7.5厘米KwK 37炮
no info box https://www.wanweibaike.com/wiki-7.5厘米Pak 41反坦克炮
no info box https://www.wanweibaike.com/wiki-5厘米KwK 38炮
no info box https://www.wanweibaike.com/wiki-3.7厘米KwK 36炮
no info box https://www.wanweibaike.com/wiki-7.5釐米 Pak 39炮
no info box https://www.wanweibaike.com/wiki-Kfz 13
no info box https://www.wanweibaike.com/wiki-42型150公厘自走火箭炮
no info box https://www.wanweibaike.com/wiki-Zimmerit
no info box https://www.wanweibaike.com/wiki-User-FFX0324/沙盒
no info box https://www.wanweibaike.com/wiki-Wurfrahmen 40多管火箭炮
no info box ht

no info box https://www.wanweibaike.com/wiki-金光侠
no info box https://www.wanweibaike.com/wiki-金一
no info box https://www.wanweibaike.com/wiki-李斗益
no info box https://www.wanweibaike.com/wiki-胡真一
no info box https://www.wanweibaike.com/wiki-陈雷 (1917年)
no info box https://www.wanweibaike.com/wiki-全昌哲
no info box https://www.wanweibaike.com/wiki-崔春國
no info box https://www.wanweibaike.com/wiki-崔光 (1918年)
no info box https://www.wanweibaike.com/wiki-柳京洙
no info box https://www.wanweibaike.com/wiki-白鹤林 (朝鲜)
no info box https://www.wanweibaike.com/wiki-王明贵
no info box https://www.wanweibaike.com/wiki-朴洛權
no info box https://www.wanweibaike.com/wiki-德占波兰境内的犹太人隔都
no info box https://www.wanweibaike.com/wiki-卡齐米日 (克拉科夫)
no info box https://www.wanweibaike.com/wiki-阿多内·佐利
no info box https://www.wanweibaike.com/wiki-恩里科·贝林格
no info box https://www.wanweibaike.com/wiki-埃利奧·維托里尼
no info box https://www.wanweibaike.com/wiki-艾利萨·巴兹纳
no info box https://www.wanweibaike.com/wiki-纳科·斯皮鲁
no info box htt